# Classification models with PySpark

In this notebook we will learn more about classification models using Pyspark library. The main objective is to create a classification models that identifies which clients wants to cancel the service or not.

## Preparing Data

In this section we will be preparing data to feed our model, also an analysis will be done.

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark= SparkSession.builder.master('local[*]').getOrCreate()

spark

In [16]:
import zipfile

with zipfile.ZipFile('base de dados.zip', 'r') as zip:
    zip.extractall()

dados = spark.read.csv("base de dados\dados_clientes.csv", sep=',', header=True, inferSchema=True)

In [17]:
dados

DataFrame[id: int, Churn: string, Mais65anos: int, Conjuge: string, Dependentes: string, MesesDeContrato: int, TelefoneFixo: string, MaisDeUmaLinhaTelefonica: string, Internet: string, SegurancaOnline: string, BackupOnline: string, SeguroDispositivo: string, SuporteTecnico: string, TVaCabo: string, StreamingFilmes: string, TipoContrato: string, ContaCorreio: string, MetodoPagamento: string, MesesCobrados: double]

In [9]:
dados.limit(20).toPandas()

,id,Churn,Mais65anos,Conjuge,Dependentes,MesesDeContrato,TelefoneFixo,MaisDeUmaLinhaTelefonica,Internet,SegurancaOnline,BackupOnline,SeguroDispositivo,SuporteTecnico,TVaCabo,StreamingFilmes,TipoContrato,ContaCorreio,MetodoPagamento,MesesCobrados
0,0,Nao,0,Sim,Nao,1,Nao,SemServicoTelefonico,DSL,Nao,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,BoletoEletronico,29.85
1,1,Nao,0,Nao,Nao,34,Sim,Nao,DSL,Sim,Nao,Sim,Nao,Nao,Nao,UmAno,Nao,Boleto,56.95
2,2,Sim,0,Nao,Nao,2,Sim,Nao,DSL,Sim,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,Boleto,53.85
3,3,Nao,0,Nao,Nao,45,Nao,SemServicoTelefonico,DSL,Sim,Nao,Sim,Sim,Nao,Nao,UmAno,Nao,DebitoEmConta,42.30
4,4,Sim,0,Nao,Nao,2,Sim,Nao,FibraOptica,Nao,Nao,Nao,Nao,Nao,Nao,Mensalmente,Sim,BoletoEletronico,70.70
5,5,Sim,0,Nao,Nao,8,Sim,Sim,FibraOptica,Nao,Nao,Sim,Nao,Sim,Sim,Mensalmente,Sim,BoletoEletronico,99.65
6,6,Nao,0,Nao,Sim,22,Sim,Sim,FibraOptica,Nao,Sim,Nao,Nao,Sim,Nao,Mensalmente,Sim,CartaoCredito,89.10
7,7,Nao,0,Nao,Nao,10,Nao,SemServicoTelefonico,DSL,Sim,Nao,Nao,Nao,Nao,Nao,Mensalmente,Nao,Boleto,29.75
8,8,Sim,0,Sim,Nao,28,Sim,Sim,FibraOptica,Nao,Nao,Sim,Sim,Sim,Sim,Mensalmente,Sim,BoletoEletronico,104.80
9,9,Nao,0,Nao,Sim,62,Sim,Nao,DSL,Sim,Sim,Nao,Nao,Nao,Nao,UmAno,Nao,DebitoEmConta,56.15


In [5]:
dados.count()

10348

In [6]:
dados.groupBy('Churn').count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|  Sim| 5174|
|  Nao| 5174|
+-----+-----+



In [7]:
dados.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Churn: string (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- Conjuge: string (nullable = true)
 |-- Dependentes: string (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- TelefoneFixo: string (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: string (nullable = true)
 |-- Internet: string (nullable = true)
 |-- SegurancaOnline: string (nullable = true)
 |-- BackupOnline: string (nullable = true)
 |-- SeguroDispositivo: string (nullable = true)
 |-- SuporteTecnico: string (nullable = true)
 |-- TVaCabo: string (nullable = true)
 |-- StreamingFilmes: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- ContaCorreio: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)



In [8]:
dados.groupBy('Internet').count().show()

+-----------+-----+
|   Internet|count|
+-----------+-----+
|FibraOptica| 5401|
|        Nao| 1741|
|        DSL| 3206|
+-----------+-----+



# Data Treatment

In this section we will treat data, change some of the types and change words for binary or integers to represent categories.

In [18]:
colunas_binarias=[
    'Churn',
    'Conjuge',
    'TelefoneFixo',
    'MaisDeUmaLinhaTelefonica',
    'SegurancaOnline',
    'SeguroDispositivo',
    'SuporteTecnico',
    'TVaCabo',
    'StreamingFilmes',
    'ContaCorreio'
]

In [19]:
from pyspark.sql import functions as f

In [20]:
todas_colunas= [f.when(f.col(c) == 'Sim', 1).otherwise(0).alias(c) for c in colunas_binarias]

In [21]:
for coluna in reversed(dados.columns):
    if coluna not in colunas_binarias:
        todas_colunas.insert(0, coluna)
todas_colunas

['id',
 'Mais65anos',
 'Dependentes',
 'MesesDeContrato',
 'Internet',
 'BackupOnline',
 'TipoContrato',
 'MetodoPagamento',
 'MesesCobrados',
 Column<'CASE WHEN (Churn = Sim) THEN 1 ELSE 0 END AS `Churn`'>,
 Column<'CASE WHEN (Conjuge = Sim) THEN 1 ELSE 0 END AS `Conjuge`'>,
 Column<'CASE WHEN (TelefoneFixo = Sim) THEN 1 ELSE 0 END AS `TelefoneFixo`'>,
 Column<'CASE WHEN (MaisDeUmaLinhaTelefonica = Sim) THEN 1 ELSE 0 END AS `MaisDeUmaLinhaTelefonica`'>,
 Column<'CASE WHEN (SegurancaOnline = Sim) THEN 1 ELSE 0 END AS `SegurancaOnline`'>,
 Column<'CASE WHEN (SeguroDispositivo = Sim) THEN 1 ELSE 0 END AS `SeguroDispositivo`'>,
 Column<'CASE WHEN (SuporteTecnico = Sim) THEN 1 ELSE 0 END AS `SuporteTecnico`'>,
 Column<'CASE WHEN (TVaCabo = Sim) THEN 1 ELSE 0 END AS `TVaCabo`'>,
 Column<'CASE WHEN (StreamingFilmes = Sim) THEN 1 ELSE 0 END AS `StreamingFilmes`'>,
 Column<'CASE WHEN (ContaCorreio = Sim) THEN 1 ELSE 0 END AS `ContaCorreio`'>]

In [22]:
dados.select(
    todas_colunas
).show()

+---+----------+-----------+---------------+-----------+------------------+------------+----------------+-------------+-----+-------+------------+------------------------+---------------+-----------------+--------------+-------+---------------+------------+
| id|Mais65anos|Dependentes|MesesDeContrato|   Internet|      BackupOnline|TipoContrato| MetodoPagamento|MesesCobrados|Churn|Conjuge|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|
+---+----------+-----------+---------------+-----------+------------------+------------+----------------+-------------+-----+-------+------------+------------------------+---------------+-----------------+--------------+-------+---------------+------------+
|  0|         0|        Nao|              1|        DSL|               Sim| Mensalmente|BoletoEletronico|        29.85|    0|      1|           0|                       0|              0|                0|             0|      

In [23]:
dataset= dados.select(todas_colunas)

In [24]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- Dependentes: string (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- Internet: string (nullable = true)
 |-- BackupOnline: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)

